In [1]:
import pandas as pd
from pyvis.network import Network
import networkx as nx
from IPython.display import HTML

In [2]:
df = pd.read_csv('synthetic_data_PREFIX.csv')
df.head()

,ts,uid,synth_src_ip,src_port,synth_dest_ip,dest_port,proto,service,duration,orig_bytes,...,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,orig_mac,resp_mac,prefix_src,prefix_dest
0,1.696677e+09,CrOwKi4m4j7TduMCEi,169.228.0.0,57550,127.0.0.200,53,udp,dns,0.001413,43,...,Dd,1,71,2,270,(empty),-,-,UCSD - 169,Loopback
1,1.696678e+09,CiR0D53r6d7EpKJC17,132.239.0.200,61250,172.16.1.244,7000,tcp,-,2.003713,0,...,S,4,256,0,0,(empty),-,-,UCSD - 132,Private network range - 172
2,1.696678e+09,CJmsBd12kiY9BRnSi8,127.0.1.244,60872,192.168.1.244,53,udp,dns,0.000224,34,...,Dd,1,62,2,228,(empty),-,-,Loopback,Private network range - 192
3,1.696677e+09,CF3ugp1tyx18akKCE8,100.64.0.200,50466,192.168.1.244,7000,tcp,-,4.110951,0,...,S,10,640,0,0,(empty),-,-,CGNAT,Private network range - 192
4,1.696678e+09,CwdeEK3IQhUNPi8Nwh,0.0.1.244,51127,192.135.238.10,53,tcp,dns,0.005200,36,...,ShADadFf,6,288,10,596,(empty),-,-,This network,Marine Physical Lab - ..238


In [8]:
pd.concat([df['prefix_src'], df['prefix_dest']]).unique().tolist()

['UCSD - 169',
 'UCSD - 132',
 'Loopback',
 'CGNAT',
 'This network',
 'Private network range',
 'Private network range - 172',
 'UCSD - 137',
 'UCSD - 128',
 'Marine Physical Lab - ..238',
 'Private network range - 192',
 'Marine Physical Lab - ..237']

In [6]:
prefixes = {}
for index, row in df.iterrows():
    src = str(row['synth_src_ip'])
    dest = str(row['synth_dest_ip'])
    src_prefix = str(row['prefix_src'])
    dest_prefix = str(row['prefix_dest'])
    #print(f'checking {src}...')
    if src in prefixes:
        if prefixes[src] != src_prefix:
            print(f'{src}: expected {prefixes[src]} but got {src_prefix}')
    else:
        prefixes[src] = src_prefix
    #print(f'checking {dest}...')
    if dest in prefixes:
        if prefixes[dest] != dest_prefix:
            print(f'{dest}: expected {prefixes[dest]} but got {dest_prefix}')
    else:
        prefixes[dest] = dest_prefix
print(prefixes)

{'169.228.0.0': 'UCSD - 169', '127.0.0.200': 'Loopback', '132.239.0.200': 'UCSD - 132', '172.16.1.244': 'Private network range - 172', '127.0.1.244': 'Loopback', '192.168.1.244': 'Private network range - 192', '100.64.0.200': 'CGNAT', '0.0.1.244': 'This network', '192.135.238.10': 'Marine Physical Lab - ..238', '137.110.1.244': 'UCSD - 137', '132.239.0.50': 'UCSD - 132', '137.110.0.0': 'UCSD - 137', '10.0.1.244': 'Private network range', '0.0.0.200': 'This network', '10.0.0.200': 'Private network range', '10.0.0.0': 'Private network range', '172.16.0.200': 'Private network range - 172', '128.54.0.0': 'UCSD - 128', '169.228.1.244': 'UCSD - 169', '192.135.238.3': 'Marine Physical Lab - ..238', '172.16.0.0': 'Private network range - 172', '192.168.0.50': 'Private network range - 192', '0.0.0.0': 'This network', '137.110.0.200': 'UCSD - 137', '169.228.0.200': 'UCSD - 169', '10.0.0.50': 'Private network range', '137.110.0.50': 'UCSD - 137', '128.54.0.200': 'UCSD - 128', '100.64.0.0': 'CGNAT

In [4]:
relevant_properties = ['synth_src_ip', 'synth_dest_ip', 'conn_state', 'orig_bytes']
df_relevant = df[relevant_properties]
df_relevant

,synth_src_ip,synth_dest_ip,conn_state,orig_bytes
0,169.228.0.0,127.0.0.200,SF,43
1,132.239.0.200,172.16.1.244,S0,0
2,127.0.1.244,192.168.1.244,SF,34
3,100.64.0.200,192.168.1.244,S0,0
4,0.0.1.244,192.135.238.10,SF,36
...,...,...,...,...
986,0.0.0.200,0.0.0.0,S0,0
987,128.54.0.200,127.0.0.0,S0,0
988,128.54.1.244,137.110.0.50,SF,2925
989,192.168.0.50,100.64.0.50,S0,0


In [40]:
nodes = pd.concat([df_relevant['synth_src_ip'], df_relevant['synth_dest_ip']]).unique().tolist()
